In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import cv2, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import torchvision
from torchvision import transforms, datasets, models
from glob import glob
!pip install torch_summary

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d tongpython/cat-and-dog
!unzip cat-and-dog.zip

In [ ]:
#@ Creating training and testing folder:
train_data_dir='/content/training_set/training_set'
test_data_dir='/content/test_set/test_set'

In [ ]:
from torch.utils.data import DataLoader, Dataset
from random import shuffle, seed

seed(22)

class cats_dogs(Dataset):
  def __init__(self, folder):
    cats=glob(folder + '/cats/*.jpg')
    dogs=glob(folder+'/dogs/*.jpg')
    self.fpaths=cats + dogs
    shuffle(self.fpaths)
    self.targets=[fpath.split('/')[-1].startswith('dog') for fpath in self.fpaths] #dog=1

  def __len__(self):
    return len(self.fpaths)

  def __getitem__(self, ix):
    f=self.fpaths[ix]
    target=self.targets[ix]
    im=(cv2.imread(f)[:, :, ::-1]) #BGR -> RGB
    im=cv2.resize(im, (224, 224))
    return torch.tensor(im/255.).permute(2, 0, 1).to(device).float(), \
    torch.tensor([target]).float().to(device) #permute: (H, W, C) -> (C, H, W)


In [ ]:
def conv_layer(n_inputs, n_outputs, kernel_size, stride=1):
  return nn.Sequential(
      nn.Conv2d(n_inputs, n_outputs, kernel_size, stride),
      nn.ReLU(),
      nn.BatchNorm2d(n_outputs),
      nn.MaxPool2d(2)
  )

In [ ]:
def get_model():
  model=nn.Sequential(
      conv_layer(3, 64, 3),
      conv_layer(64, 512, 3),
      conv_layer(512, 512, 3),
      conv_layer(512, 512, 3),
      conv_layer(512, 512, 3),
      conv_layer(512, 512, 3),
      nn.Flatten(),
      nn.Linear(512, 1),
      nn.Sigmoid()
  ).to(device)

  loss_func=nn.BCELoss()
  optimizer=torch.optim.Adam(model.parameters(), lr=1e-3)
  return model, loss_func, optimizer

In [ ]:
def get_data():
  train=cats_dogs(train_data_dir)
  train_dl=DataLoader(train, batch_size=32, shuffle=True, drop_last=True)
  val=cats_dogs(test_data_dir)
  val_dl=DataLoader(val, batch_size=32, shuffle=True, drop_last=True)
  return train_dl, val_dl

In [ ]:
def train_batch(x, y, model, opt, loss_func):
  model.train()
  prediction=model(x)
  batch_loss=loss_func(prediction, y)
  batch_loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  return batch_loss.item()

In [ ]:
@torch.no_grad()
def accuracy(x, y, model):
  prediction=model(x)
  is_correct=(prediction > 0.5) == y
  return is_correct.cpu().numpy().tolist()

In [ ]:
@torch.no_grad()
def val_loss(x, y, model):
  prediction=model(x)
  val_loss=loss_func(prediction, y)
  return val_loss.item()